In [ ]:
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import ImageFolder
from torchvision import transforms

from PIL import Image

import cv2

import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import re
import os
import numpy as np

In [ ]:
!pip install torch
!pip install torchvision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
!unzip "/content/drive/MyDrive/Driver Drowsiness Dataset (DDD).zip" -d "/content/drive/MyDrive/newdata/"

Streaming output truncated to the last 5000 lines.
  inflating: /content/drive/MyDrive/newdata/Driver Drowsiness Dataset (DDD)/Non Drowsy/y0082.png  
  inflating: /content/drive/MyDrive/newdata/Driver Drowsiness Dataset (DDD)/Non Drowsy/y0083.png  
  inflating: /content/drive/MyDrive/newdata/Driver Drowsiness Dataset (DDD)/Non Drowsy/y0084.png  
  inflating: /content/drive/MyDrive/newdata/Driver Drowsiness Dataset (DDD)/Non Drowsy/y0085.png  
  inflating: /content/drive/MyDrive/newdata/Driver Drowsiness Dataset (DDD)/Non Drowsy/y0086.png  
  inflating: /content/drive/MyDrive/newdata/Driver Drowsiness Dataset (DDD)/Non Drowsy/y0087.png  
  inflating: /content/drive/MyDrive/newdata/Driver Drowsiness Dataset (DDD)/Non Drowsy/y0088.png  
  inflating: /content/drive/MyDrive/newdata/Driver Drowsiness Dataset (DDD)/Non Drowsy/y0089.png  
  inflating: /content/drive/MyDrive/newdata/Driver Drowsiness Dataset (DDD)/Non Drowsy/y0090.png  
  inflating: /content/drive/MyDrive/newdata/Driver Drowsin

In [ ]:
from google.colab.patches import cv2_imshow

im_unchanged = cv2.imread("/content/drive/MyDrive/Driver Drowsiness Dataset (DDD)/Drowsy/B0002.png",cv2.IMREAD_UNCHANGED)
im_rgb = cv2.cvtColor(im_unchanged, cv2.COLOR_BGR2RGB)
#cv2_imshow(arr)

print("im_rgb cv2")
print(im_rgb)
#print(np.array(arr))


arr2 = Image.open("/content/drive/MyDrive/Driver Drowsiness Dataset (DDD)/Drowsy/B0002.png").convert('RGB')

#arr3 = np.array(arr2)
#print(arr == arr3)

print("arr2 Image")
#print(np.array(arr2))
print(arr2)


print('== ?')
print(im_rgb == arr2)

im_rgb cv2
[[[253 254 151]
  [252 255 151]
  [250 254 151]
  ...
  [255 232 147]
  [254 231 142]
  [251 228 138]]

 [[253 254 151]
  [252 255 151]
  [251 255 152]
  ...
  [250 229 142]
  [255 235 145]
  [253 230 140]]

 [[252 254 151]
  [251 255 151]
  [251 255 151]
  ...
  [248 226 141]
  [255 235 146]
  [253 230 140]]

 ...

 [[ 47  45  46]
  [ 47  46  45]
  [ 46  46  45]
  ...
  [105  88  78]
  [ 71  57  52]
  [ 48  37  34]]

 [[ 50  47  47]
  [ 50  48  47]
  [ 49  48  46]
  ...
  [103  86  77]
  [ 70  54  50]
  [ 49  36  33]]

 [[ 59  54  51]
  [ 59  54  51]
  [ 59  54  51]
  ...
  [100  80  74]
  [ 62  46  44]
  [ 46  33  31]]]
arr2 Image
<PIL.Image.Image image mode=RGB size=227x227 at 0x7FB9EBE15C70>
== ?
[[[ True  True  True]
  [ True  True  True]
  [ True  True  True]
  ...
  [ True  True  True]
  [ True  True  True]
  [ True  True  True]]

 [[ True  True  True]
  [ True  True  True]
  [ True  True  True]
  ...
  [ True  True  True]
  [ True  True  True]
  [ True  True  True]]


In [6]:
# Add image paths, labels, file names and person IDs into lists
image_path, label, file_name, person_ID = [], [], [], []

pattern = re.compile(r'^[a-zA-Z][a-zA-Z]?') #identification criteria of participant

data_dir = "/content/drive/MyDrive/newdata/Driver Drowsiness Dataset (DDD)/"

for class_name in os.listdir(data_dir):  #goes through all folders in DDD folder
  for path in os.listdir(data_dir+class_name):  #goes through all files in both folders and updates file path and labels
    if class_name == 'Drowsy':
      label.append(0)
    else:
      label.append(1)
    image_path.append(os.path.join(data_dir, class_name, path))
    file_name.append(path)

    person_ID.append(pattern.findall(path)[0])  #identification of participant
  
df = pd.DataFrame()
df['images'] = image_path
df['label'] = label
df['name'] = file_name
df['person'] = person_ID
df = df.sample(frac=1).reset_index(drop=True)
df.head(100)



,images,label,name,person
0,/content/drive/MyDrive/newdata/Driver Drowsine...,0,W1063.png,W
1,/content/drive/MyDrive/newdata/Driver Drowsine...,0,T0090.png,T
2,/content/drive/MyDrive/newdata/Driver Drowsine...,0,W0685.png,W
3,/content/drive/MyDrive/newdata/Driver Drowsine...,0,K0569.png,K
4,/content/drive/MyDrive/newdata/Driver Drowsine...,1,i0545.png,i
...,...,...,...,...
95,/content/drive/MyDrive/newdata/Driver Drowsine...,1,i0996.png,i
96,/content/drive/MyDrive/newdata/Driver Drowsine...,1,zb0135.png,zb
97,/content/drive/MyDrive/newdata/Driver Drowsine...,0,G0119.png,G
98,/content/drive/MyDrive/newdata/Driver Drowsine...,1,v0348.png,v


In [ ]:
print(np.array(image_path)) #.reshape([-1,1])
print("--")
print(np.array(image_path).reshape([-1,1]))

['Driver Drowsiness Dataset (DDD)/Drowsy/ZC0332.png'
 'Driver Drowsiness Dataset (DDD)/Drowsy/ZC0330.png'
 'Driver Drowsiness Dataset (DDD)/Drowsy/ZC0329.png' ...
 'Driver Drowsiness Dataset (DDD)/Non Drowsy/a0458.png'
 'Driver Drowsiness Dataset (DDD)/Non Drowsy/a0461.png'
 'Driver Drowsiness Dataset (DDD)/Non Drowsy/a0446.png']
--
[['Driver Drowsiness Dataset (DDD)/Drowsy/ZC0332.png']
 ['Driver Drowsiness Dataset (DDD)/Drowsy/ZC0330.png']
 ['Driver Drowsiness Dataset (DDD)/Drowsy/ZC0329.png']
 ...
 ['Driver Drowsiness Dataset (DDD)/Non Drowsy/a0458.png']
 ['Driver Drowsiness Dataset (DDD)/Non Drowsy/a0461.png']
 ['Driver Drowsiness Dataset (DDD)/Non Drowsy/a0446.png']]


In [7]:
from sklearn.model_selection import train_test_split
image_path = np.array(image_path).reshape([-1,1])
label = np.array(label).reshape([-1,1])
comp_data = np.hstack((image_path, label))
#print(comp_data)

train, test = train_test_split(comp_data, test_size=0.2, random_state=1234)
#print(train)

In [8]:
transformation = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,),(0.5,))
])

#completeDataset = ImageFolder(root=data_dir, transform=transformation)

class DDDdataset(Dataset):
  def __init__(self, image_paths, image_labels, transform):
    super().__init__()
    self.paths = image_paths
    self.labels = image_labels
    self.len = len(self.paths)
    self.transform = transform
  
  def __len__(self): return self.len
  
  def __getitem__(self, index):
    path = self.paths[index]
    label = self.labels[index]
    target_tensor=torch.from_numpy(np.array(label,dtype=np.int16))
    #image = Image.open(path).convert('RGB')
    image = cv2.imread(path,cv2.IMREAD_UNCHANGED)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = self.transform(image)
    return (image, target_tensor)

trainDataset = DDDdataset(train[:,0], train[:,-1], transformation)
testDataset = DDDdataset(test[:,0], test[:,-1], transformation)

trainDataLoader = DataLoader(trainDataset, batch_size=50, shuffle=True)
testDataLoader = DataLoader(testDataset, batch_size=50, shuffle=True)

In [9]:
class CNN(nn.Module):
  def __init__(self):
    super().__init__()

    self.conv1 = nn.Conv2d(in_channels=3, out_channels=10, kernel_size=(4,4), stride=1, padding=1)
    self.conv2 = nn.Conv2d(in_channels=10, out_channels=20, kernel_size=(5,5), stride=2, padding=1)
    self.conv3 = nn.Conv2d(in_channels=20, out_channels=32, kernel_size=(6,6), stride=2, padding=1)
    #self.conv4 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3,3), stride=1, padding=1)
    self.conv5 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3,3), stride=1, padding=1)

    self.fc1 = nn.Linear(in_features=64*3*3, out_features=250)
    self.fc2 = nn.Linear(in_features=250, out_features=50)
    self.fc3 = nn.Linear(in_features=50, out_features=2)

  def forward(self, X):

    X = F.relu(self.conv1(X))
    X = F.max_pool2d(X, 2)

    X = F.relu(self.conv2(X))
    X = F.max_pool2d(X, 2)

    X = F.relu(self.conv3(X))
    X = F.max_pool2d(X, 3, stride=2)

    #X = F.relu(self.conv4(X))
    #X = F.max_pool2d(X, 2)

    X = F.relu(self.conv5(X))
    X = F.max_pool2d(X, 2)

    X = X.view(X.shape[0], -1)
    X = F.relu(self.fc1(X))
    X = F.relu(self.fc2(X))
    X = self.fc3(X)

    return X




In [11]:
import time

model = CNN()
number_epochs = 3
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
losses = []
accuracies = []



start = time.time()
for epoch in range(number_epochs):

  #extra
  epoch_loss = 0
  epoch_accuracy = 0

  for (X, y) in trainDataLoader:
    predictions = model(X)
    loss = loss_func(predictions, y.long())

    optimizer.zero_grad()
    loss.backward()

    optimizer.step()

    #extra
    accuracy = ((predictions.argmax(dim=1) == y).float().mean())
    epoch_accuracy += accuracy
    epoch_loss += loss
  
  #extra
  epoch_accuracy = epoch_accuracy/len(trainDataLoader)
  accuracies.append(epoch_accuracy)
  epoch_loss = epoch_loss / len(trainDataLoader)
  losses.append(epoch_loss)

  print("\n --- Epoch: {}, train loss: {:.4f}, train acc: {:.4f}, time: {}".format(epoch, epoch_loss, epoch_accuracy, time.time() - start))

  model.eval()
  with torch.no_grad():
    test_epoch_loss = 0
    test_epoch_accuracy = 0

    for test_X, test_y in testDataLoader:

      test_preds = model(test_X)
      test_loss = loss_func(test_preds, test_y.long())

      test_epoch_loss += test_loss            
      test_accuracy = ((test_preds.argmax(dim=1) == test_y).float().mean())
      test_epoch_accuracy += test_accuracy

    test_epoch_accuracy = test_epoch_accuracy/len(testDataLoader)
    test_epoch_loss = test_epoch_loss / len(testDataLoader)

    print("Epoch: {}, test loss: {:.4f}, test acc: {:.4f}, time: {}\n".format(epoch, test_epoch_loss, test_epoch_accuracy, time.time() - start))
  model.train(mode=True)



 --- Epoch: 0, train loss: 0.0865, train acc: 0.9634, time: 803.8206884860992
Epoch: 0, test loss: 0.0127, test acc: 0.9971, time: 939.3580636978149


 --- Epoch: 1, train loss: 0.0106, train acc: 0.9968, time: 1699.6715869903564
Epoch: 1, test loss: 0.0071, test acc: 0.9982, time: 1821.7067024707794


 --- Epoch: 2, train loss: 0.0085, train acc: 0.9975, time: 2563.4713139533997
Epoch: 2, test loss: 0.0119, test acc: 0.9972, time: 2687.953735589981

